In [ ]:
import pandas as pd
from numpy import nan as NA
import numpy as np

Get the filename to be loaded.

In [ ]:
file_name = str(input("enter the name of the file: "))

Load file from input.

In [ ]:
data_combined = pd.concat(pd.read_excel(file_name + '.xlsx',ignore_index=True, sheet_name=None, header=26))
data_raw = pd.read_excel(file_name + '.xlsx', sheet_name=None, ignore_index=True)

Create a list with the sheetnames.

In [ ]:
sheet_list = list(data_raw.keys())

Create a list containing only the movements (and thus number of movements) from the sheetnames.

In [ ]:
movement_list = []
for sheet in sheet_list:
        if 'Movement' in sheet:
            movement_list.append(sheet)
number_of_movements = len(movement_list)

Create a series containing only the PCU totals, per 15 minute interval.

In [ ]:
remove_variables = ['Hourly Total', 'Hourly Average', 'Session Total', 'Session Average', 'TIME']

for i in remove_variables:
    data_combined = data_combined.drop(data_combined[(data_combined['TIME'] == i)].index)
    
cleaned_PCU_totals = data_combined['PCU TOTAL'].dropna()

Reshape the series as an array of columns as 15 minute time intervals and rows as the movements.

In [ ]:
arr_totals = np.array(cleaned_PCU_totals)
number_of_time_intervals = len(arr_totals)/number_of_movements
organised_PCU_totals = arr_totals.reshape(number_of_movements, int(number_of_time_intervals))

Identify the indices at which to split the array, such that AM, inter and PM peaks can be found.

In [ ]:
number_of_time_intervals = int(input("Enter the number of intervals: "))
split_locations = []
split_index = 0
for interval in range(number_of_time_intervals-1):
    split_index = (int(input("How many hours are there in interval " + str(interval+1) + ": "))*4) + split_index
    split_locations.append(split_index)

survey_intervals = np.hsplit(organised_PCU_totals,split_locations)

Total the columns (15 minute intervals), then calculate the rolling hour sum (Four consequetive columns). Drop null values which are the first three times.

In [ ]:
rolling_hour_totals =[]
for i in survey_intervals:
    quarterly_sum = np.sum(i , axis=0)
    quarterly_sum_df = pd.DataFrame(quarterly_sum)
    rolling_hour_totals.append(quarterly_sum_df.rolling(4,).sum().dropna())

Display rolling hour PCU total, for the 15 minute intervals in the data, the first non-zero value is the first time interval in the data.

In [ ]:
print( "The rolling hour totals, per 15 minute time interval (in PCUs) are: \n" + str(rolling_hour_totals))

Get the maximum values, i.e. the peaks. 

In [ ]:
print("The peak values are: ")
for survey in rolling_hour_totals:
    x=np.array(survey)
    print(x.max())